In [6]:
!pip install -q \
    transformers \
    accelerate \
    peft \
    bitsandbytes \
    datasets \
    sentence-transformers \
    faiss-cpu \
    pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 92.2 MB/s eta 0:00:00:00:0100:01


In [8]:
import torch
import faiss
import json
import numpy as np
import fitz
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model

2025-12-17 18:39:55.941196: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765996796.121744      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765996796.179062      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765996796.627021      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765996796.627059      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765996796.627062      55 computation_placer.cc:177] computation placer alr

In [9]:
MODEL_NAME = "Qwen/Qwen3-4B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [10]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 2,949,120 || all params: 4,025,417,216 || trainable%: 0.0733


In [14]:
dataset = load_dataset(
    "json",
    data_files="/kaggle/input/lora-train-json/lora_train.jsonl"
)

def tokenize(example):
    messages = [
        {"role": "user", "content": example["instruction"]},
        {"role": "assistant", "content": example["response"]}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )

    out = tokenizer(
        text,
        truncation=True,
        max_length=512
    )
    out["labels"] = out["input_ids"].copy()
    return out

dataset = dataset.map(tokenize, remove_columns=dataset["train"].column_names)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

In [15]:
args = TrainingArguments(
    output_dir="lora_out",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    logging_steps=10,
    save_strategy="epoch",
    optim="paged_adamw_8bit",
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"]
)

trainer.train()

Step,Training Loss
10,1.910300
20,1.639400
30,1.594500
40,1.424100
50,1.486100
60,1.333800


TrainOutput(global_step=64, training_loss=1.556925855576992, metrics={'train_runtime': 212.4014, 'train_samples_per_second': 1.205, 'train_steps_per_second': 0.301, 'total_flos': 2236379911907328.0, 'train_loss': 1.556925855576992, 'epoch': 2.0})

In [16]:
model.save_pretrained("qwen3-4b-course")
tokenizer.save_pretrained("qwen3-4b-course")

('qwen3-4b-course/tokenizer_config.json',
 'qwen3-4b-course/special_tokens_map.json',
 'qwen3-4b-course/chat_template.jinja',
 'qwen3-4b-course/vocab.json',
 'qwen3-4b-course/merges.txt',
 'qwen3-4b-course/added_tokens.json',
 'qwen3-4b-course/tokenizer.json')

In [18]:
index = faiss.read_index("/kaggle/input/vectorstore/index.faiss")
texts = np.load("/kaggle/input/vectorstore/texts.npy", allow_pickle=True)

embedder = SentenceTransformer("intfloat/e5-base-v2")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [19]:
def youtube_search(q):
    return f"https://www.youtube.com/results?search_query={q.replace(' ', '+')}"

def arxiv_search(q):
    return f"https://arxiv.org/search/?query={q.replace(' ', '+')}&searchtype=all"

In [25]:
def retrieve_context(question, k=4):
    emb = embedder.encode([question]).astype("float32")
    _, idx = index.search(emb, k)
    return "\n\n".join(texts[i] for i in idx[0])

In [26]:
def generate_answer(question):
    context = retrieve_context(question)

    messages = [
        {
            "role": "system",
            "content": "You are an IIIT course assistant."
        },
        {
            "role": "user",
            "content": f"""
Context:
{context}

Question:
{question}

Answer clearly, completely, and in a structured manner.
"""
        }
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=2048,
        temperature=0.6,
        top_p=0.95
    )

    answer = tokenizer.decode(output[0], skip_special_tokens=True)

    # ✅ ADD THESE
    yt_links = youtube_search(question)
    arxiv_links = arxiv_search(question)

    # Safety: ensure lists
    if isinstance(yt_links, str):
        yt_links = [yt_links]
    if isinstance(arxiv_links, str):
        arxiv_links = [arxiv_links]

    return {
        "answer": answer,
        "youtube": yt_links,
        "papers": arxiv_links
    }

# Validation

In [27]:
# 🔍 Trial test cell — verify RAG + LoRA model works end-to-end
test_question = "What is a random variable? Explain with examples."

print("=" * 70)
print(" IIIT Course Assistant – Trial RAG Test")
print("=" * 70)

print("\n🧑‍🎓 Question:")
print(test_question)

print("\n🔍 Thinking...\n")

result = generate_answer(test_question)

print("📘 Answer:")
print("-" * 70)
print(result["answer"].strip())
print("-" * 70)

if result.get("youtube"):
    print("\n📺 Suggested YouTube Videos:")
    for link in result["youtube"]:
        print("  •", link)

if result.get("papers"):
    print("\n📄 Relevant Research Papers:")
    for paper in result["papers"]:
        print("  •", paper)

print("\n" + "=" * 70)
print("✅ Trial test completed successfully")

 IIIT Course Assistant – Trial RAG Test

🧑‍🎓 Question:
What is a random variable? Explain with examples.

🔍 Thinking...

📘 Answer:
----------------------------------------------------------------------
system
You are an IIIT course assistant.
user

Context:
Continuous random variables
▶A random variable defined on R is discrete, if FX(·) is
piecewise constant.
▶A random variable defined on R is continuous, if FX(·) is a
continuous function.
▶Examples of Continuous random variables
1. Pick a number uniformly from [a, b].
2. Time interval between successive customers entering DMart.
3. Travel time from office to home.
4. Level of water in a dam or pending workload on a server.
8 / 29

Example of function of Random variables
▶Suppose X and Y are continuous independent random
variables. Let W = max(X, Y ) and Z = min(X, Y ) Find the
CDF and pdf of Z.
▶HW: When X and Y are exponential with parameters λ1 and
λ2 then Z is also exponential with parameter λ1 + λ2.
5 / 42

A running example
▶Con

# Test

In [30]:
# doc = fitz.open("/kaggle/input/endsem/endsem.pdf")
# questions = []

# for page in doc:
#     for line in page.get_text().split("\n"):
#         if line.lower().startswith(("q", "question")):
#             questions.append(line)

# results = {}
# for i, q in enumerate(questions):
#     results[f"Q{i+1}"] = generate_answer(q)

# with open("endsem_results.json", "w") as f:
#     json.dump(results, f, indent=2)
import json
import re

PDF_PATH = "/kaggle/input/endsem/endsem_solutions.pdf" 

def extract_questions_from_solutions(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text()
    
    # regex to find patterns like "Q1:", "Q2:", "(a)", "(b)" etc.
    # This splits the text based on "Q" followed by a number and a colon
    question_blocks = re.split(r'(Q\d+[:])', full_text)
    
    extracted_questions = []
    # The first element is usually header text, so we skip it
    for i in range(1, len(question_blocks), 2):
        q_marker = question_blocks[i]      # e.g., "Q1:"
        q_content = question_blocks[i+1]    # The text following it
        
        # We stop the question text before the answer "A:" begins
        actual_question = q_content.split('A:')[0].strip()
        extracted_questions.append(f"{q_marker} {actual_question}")
        
    return extracted_questions

# 1. Extract the questions
print("Reading questions from PDF...")
questions_to_test = extract_questions_from_solutions(PDF_PATH)

# 2. Run inference on each extracted question
results = {}
print(f"Found {len(questions_to_test)} questions. Starting inference...\n")

for i, q_text in enumerate(questions_to_test):
    print(f"Testing {q_text[:50]}...")
    try:
        # Using the generate_answer function from your notebook
        ans_data = generate_answer(q_text)
        results[f"Question_{i+1}"] = {
            "input_question": q_text,
            "agent_answer": ans_data["answer"],
            "suggested_resources": {
                "youtube": ans_data["youtube"],
                "papers": ans_data["papers"]
            }
        }
    except Exception as e:
        print(f"Error processing question {i+1}: {e}")

# 3. Save the agent's performance to a JSON file
output_file = "agent_test_results.json"
with open(output_file, "w") as f:
    json.dump(results, f, indent=4)

print(f"\n✅ Testing complete. Results saved to {output_file}")

Reading questions from PDF...
Found 12 questions. Starting inference...

Testing Q1: Let Z = X1 + X2 + · · · + XN, where Xi are i.i...
Testing Q2: Let Y = eX, where X ∼N(µ, σ2). Obtain the pdf ...
Testing Q1: Let D = {x1, x2, . . . , xn} denote i.i.d. sam...
Testing Q2: Consider a Gaussian random variable X with a k...
Testing Q3: Consider a sequence {Xn, n = 1, 2, 3, . . . } ...
Testing Q4: Given a Markov coin with the following transit...
Testing Q5: Let X and Y be independent random variables wi...
Testing Q1: Let D = {x1, . . . xn} denote i.i.d. samples f...
Testing Q2: Bayesian Inference/ Conjugate prior problem: S...
Testing Q3: (i) For a Markov chain, let Fii denote the pro...
Testing Q4: Gaussian: Suppose X = AZ + µ, where A is an n ...
Testing Q5: Shifted Exponential: Let X be a random variabl...

✅ Testing complete. Results saved to agent_test_results.json


In [33]:
!zip -r working.zip /kaggle/working

updating: kaggle/working/ (stored 0%)
updating: kaggle/working/.virtual_documents/ (stored 0%)
updating: kaggle/working/.virtual_documents/__notebook_source__.ipynb (deflated 59%)
updating: kaggle/working/lora_out/ (stored 0%)
updating: kaggle/working/lora_out/checkpoint-64/ (stored 0%)
updating: kaggle/working/lora_out/checkpoint-64/optimizer.pt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 7%)
updating: kaggle/working/lora_out/checkpoint-64/adapter_config.json (deflated 56%)
updating: kaggle/working/lora_out/checkpoint-64/README.md (deflated 65%)
updating: kaggle/working/lora_out/checkpoint-64/rng_state.pth (deflated 26%)
updating: kaggle/working/lora_out/checkpoint-64/scheduler.pt (deflated 62%)
updating: kaggle/working/lora_out/checkpoint-64/adapter_model.safetensors (deflated 8%)
updating: kaggle/working/lora_out/checkpoint-64/training_args.bin (deflated 53%)
updating: kaggle/working/lora_out/checkpoint-64/trainer_state.json (deflated 66%)
updating: kaggle/working/lora_out/checkpoint-64/scaler.pt (deflated 64%)
updating: kaggle/working/lora_out/checkpoint-32/ (stored 0%)
updating: kaggle/working/lora_out/checkpoint-32/optimizer.pt (deflated 7%)
updating: kaggle/working/lora_out/checkpoint-32/adapter_config.json (deflated 56%)
updating: kaggle/working/lora_out/checkpoint-32/README.md (deflated 65%)
updating: kaggle/working/lora_out/checkpoint-32/rng_state.pt